# robustness 検証

ヒートマップ作成用のデータフレームの作成

In [3]:
import sys
sys.path.append("..")
import torch
import pandas as pd
import numpy as np
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
from PIL import Image
from reserch_utils_HT import network_to_image
from models.set_model import CNN_base, D1D2_base
from data.data_loader import cnn_data_loader_cv, set_transform
from torch.utils.data import DataLoader
from matplotlib.cm import ScalarMappable
import matplotlib.colors as colors

In [5]:
parameter = {
    "BA": {
        20 : [1,2,3,4,5,6,7,8,9,10],
        30 : [1,2,3,4,5,6,7,8,9,10],
        50 : [1,2,3,4,5,6,7,8,9,10],
        70 : [1,2,3,4,5,6,7,8,9,10],
        100: [1,2,3,4,5,6,7,8,9,10,15,20,25,30],
        200: [1,2,3,4,5,6,7,8,9,10,15,20,25,30],
        300: [1,2,3,4,5,6,7,8,9,10,15,20,25,30],
        500: [1,2,3,4,5,6,7,8,9,10,15,20,25,30],
        700: [1,2,3,4,5,6,7,8,9,10,15,20,25,30],
        1000: [1,2,3,4,5,6,7,8,9,10,15,20,25,30,50,100,200],
        2000: [1,2,3,4,5,6,7,8,9,10,15,20,25,30,50,100,200]
    },
    "Growth": {
        20 : [1,2,3,4,5,6,7,8,9,10],
        30 : [1,2,3,4,5,6,7,8,9,10],
        50 : [1,2,3,4,5,6,7,8,9,10],
        70 : [1,2,3,4,5,6,7,8,9,10],
        100: [1,2,3,4,5,6,7,8,9,10,15,20,25,30],
        200: [1,2,3,4,5,6,7,8,9,10,15,20,25,30],
        300: [1,2,3,4,5,6,7,8,9,10,15,20,25,30],
        500: [1,2,3,4,5,6,7,8,9,10,15,20,25,30],
        700: [1,2,3,4,5,6,7,8,9,10,15,20,25,30],
        1000: [1,2,3,4,5,6,7,8,9,10,15,20,25,30,50,100,200],
        2000: [1,2,3,4,5,6,7,8,9,10,15,20,25,30,50,100,200]
    },
    "Attach":{
        20 : [20 * i for i in np.linspace(0.5, 25, 10)],
        30 : [30 * i for i in np.linspace(0.5, 25, 10)],
        50 : [50 * i for i in np.linspace(0.5, 25, 10)],
        70 : [70 * i for i in np.linspace(0.5, 25, 10)],
        
        100 : [100 * i for i in np.linspace(0.5, 25, 14)],
        200 : [200 * i for i in np.linspace(0.5, 25, 14)],
        300 : [300 * i for i in np.linspace(0.5, 25, 14)],
        500 : [500 * i for i in np.linspace(0.5, 25, 14)],
        700 : [700 * i for i in np.linspace(0.5, 25, 14)],
        1000 : [1000 * i for i in np.linspace(0.5, 25, 17)],
        2000 : [2000 * i for i in np.linspace(0.5, 25, 17)],
        
    },
    "Random": {
        20: np.logspace(-1.3, -0.1, 10),
        30: np.logspace(-1.3, -0.1, 10),
        50: np.logspace(-1.3, -0.1, 10),
        70: np.logspace(-1.3, -0.1, 10),
        100: np.logspace(-2, -0.7, 14),
        200: np.logspace(-2, -0.7, 14),
        300: np.logspace(-2, -0.7, 14),
        500: np.logspace(-2, -0.7, 14),
        700: np.logspace(-2, -0.7, 14),
        1000: np.logspace(-3., -0.9, 17),
        2000: np.logspace(-3., -0.9, 17)
    }
}

nodes = [20,30,50,70,100,200,300,500,700,1000,2000]

In [15]:
def load_image_to_tensor(resize, kind, n, p):
    """ データの読み込み, tensor結合 """
    transform = set_transform(resize)
    if kind == "Attach":
        p = int(p)
    for i in range(100):
        img = Image.open(f"./robustness_data_img/{kind}/{n}/{p}/{i}.png")
        if i == 0:
            data = transform(img).view(1,1,resize,resize)
        else:
            img_tensor = transform(img).view(1,1,resize,resize)
            data = torch.cat((data, img_tensor), 0)
    return data

def make_pred_df(model, data, kind, n, p, cnt):
    """ ネットワークごとの予測ラベルと尤度データ作成 """
    kind_to_label = {"BA": 0, "Attach": 1, "Growth": 2, "Random": 3}
    softmax = torch.nn.Softmax(1)
    with torch.no_grad():
        pred = softmax(model(data))
    index = pred.argmax(dim=1) # pred index

    pred_df = pd.DataFrame()
    pred_df["seed"] = np.array(range(100)) + 10000 # seed
    pred_df["node"] = n
    pred_df["parameter"] = p
    pred_df["kind"] = kind
    pred_df["true_label"] = kind_to_label[kind]
    pred_df["pred"] = index
    pred_df["probablility"] = [pred[i, idx].item() for i, idx in enumerate(index)]
    
    return pred_df

def robust_acc_df(model, resize, kind, parameter):
    kind_to_index = {"BA": 0, "Attach": 1, "Growth": 2, "Random": 3}
    transform = set_transform(resize)
    df = pd.DataFrame()
    cnt = 0
    for n in tqdm([20,30,50,70,100,200,300,500,700,1000,2000]):
        params = parameter[kind][n]
        for param_index, p in enumerate(params):
            # image to torch tensor
            data = load_image_to_tensor(resize, kind, n, p)
            pred_df = make_pred_df(model, data, kind, n, p, cnt)
            cnt += 100
            df = df.append(pred_df)
    return df


def make_heatmap(dataset_name, resize, parameter, gpu=True):
    # load model
    model = CNN_base("CNN", 4, resize)
    model_path = f"../logs/{dataset_name}/CNN/sort_{resize}_0.001/model_weight/fold0_trial0_epoch10.pth"
    if gpu:
        model.load_state_dict(torch.load(model_path))
    else:
        model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    
    # make acc dataframe
    pred_df = pd.DataFrame()
    for kind in ["BA", "Attach", "Growth", "Random"]:
        pred_df = pred_df.append(robust_acc_df(model, resize, kind, parameter))
    
    pred_df.to_csv(f"./robustness_plot/CNN_{dataset_name}_{resize}.csv")

In [16]:
for dataset_name in ["subset1", "poisson", "new_poisson", "new_parete"]:
    for resize in [50, 100, 200]:
        make_heatmap(dataset_name, resize, parameter, gpu=True)

100%|██████████| 11/11 [02:32<00:00, 13.89s/it]
